In [64]:
import pandas as pd
import numpy as np
import re


In [65]:
df = pd.read_csv(r'../../../data/data.csv')

In [66]:
df

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
0,NaN,322111,3F - UNITED FEDERATION OF DANISH WORKERS,@3f.dk,NaN,NaN
1,NaN,411480,3F - UNITED FEDERATION OF WORKERS IN DENMARK,@3f.dk,NaN,NaN
2,NaN,493788,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,NaN,NaN
3,NaN,511603,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,NaN,NaN
4,NaN,71936,AGORIA - BELGIAN FEDERATION FOR THE TECHNOLOGY...,NaN,AGORIA,NaN
...,...,...,...,...,...,...
674,NaN,480371,YRITTÄJÄT/FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,NaN,NaN
675,NaN,291768,ZAW (GERMAN ADVERTISING FEDERATION),@zaw.de,ZAW,NaN
676,NaN,526538,ZDH- CONFEDERATION OF GERMAN SKILLED CRAFTS AN...,@zdh.de,NaN,NaN
677,NaN,496681,ZDH GERMAN CONFEDERATION OF SKILLED CRAFTS AND...,@zdh.de,NaN,NaN


In [67]:
delimiters = " ", "/", "-"
regex_pattern = '|'.join(map(re.escape, delimiters))
regex_pattern

'\\ |/|\\-'

In [5]:
df['Acronym'] = df.Account.apply(lambda x: re.split(regex_pattern, x)[0])

In [68]:
df

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
0,NaN,322111,3F - UNITED FEDERATION OF DANISH WORKERS,@3f.dk,NaN,NaN
1,NaN,411480,3F - UNITED FEDERATION OF WORKERS IN DENMARK,@3f.dk,NaN,NaN
2,NaN,493788,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,NaN,NaN
3,NaN,511603,"A.I.S.E. - EUROPEAN FEDERATION FOR SOAPS, DETE...",@aise.eu,NaN,NaN
4,NaN,71936,AGORIA - BELGIAN FEDERATION FOR THE TECHNOLOGY...,NaN,AGORIA,NaN
...,...,...,...,...,...,...
674,NaN,480371,YRITTÄJÄT/FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,NaN,NaN
675,NaN,291768,ZAW (GERMAN ADVERTISING FEDERATION),@zaw.de,ZAW,NaN
676,NaN,526538,ZDH- CONFEDERATION OF GERMAN SKILLED CRAFTS AN...,@zdh.de,NaN,NaN
677,NaN,496681,ZDH GERMAN CONFEDERATION OF SKILLED CRAFTS AND...,@zdh.de,NaN,NaN


In [69]:
acc_list = df.groupby(by='Acronym')['Account'].agg(lambda x: x.value_counts(ascending=False).index[0] 
                                                    if x.value_counts(ascending=False)[0] > 1 
                                                    else x.value_counts(ascending=False).index[:])
for acc in acc_list:
    if type(acc) is str:
        acro = df.loc[(df.Account == acc)]['Acronym'].values[0]
        id = df.loc[(df.Account == acc)].AccountID.values[0]
        df.loc[(df['Acronym'] == acro), 'MasterID'] = id
    else:
        acro = df.loc[(df.Account == acc[0])]['Acronym'].values[0]
        if len(df.loc[(df['Acronym'] == acro)].AccountID.sort_values(ascending=True)) > 0:
            id = df.loc[(df['Acronym'] == acro)].AccountID.sort_values(ascending=True).reset_index(drop=True)[0]
            df.loc[(df['Acronym'] == acro), 'MasterID'] = id

In [70]:
acc_list = df.groupby(by='Email domain')['Account'].agg(lambda x: x.value_counts(ascending=False).index[0] 
                                                    if x.value_counts(ascending=False)[0] > 1 
                                                    else x.value_counts(ascending=False).index[:])
for acc in acc_list:
    if type(acc) is str:
        ed = df.loc[(df.Account == acc)]['Email domain'].values[0]
        id = df.loc[(df.Account == acc)].AccountID.values[0]
        df.loc[(df['Email domain'] == ed), 'MasterID'] = id
    else:
        ed = df.loc[(df.Account == acc[0])]['Email domain'].values[0]
        id = df.loc[(df['Email domain'] == ed)].AccountID.sort_values(ascending=True).reset_index(drop=True)[0]
        df.loc[(df['Email domain'] == ed), 'MasterID'] = id
        df.loc[(df['Account'] == acc[0])]

In [71]:
df.MasterID = df.MasterID.astype(np.int64)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [77]:
df.loc[df['Email domain'] == '@yrittajat.fi']

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
400,NaN,443307,FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,NaN,NaN
402,NaN,493740,FEDERATION OF FINNISH ENTERPRISES/ YRITTÄJÄT,@yrittajat.fi,NaN,NaN
674,NaN,480371,YRITTÄJÄT/FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,NaN,NaN


In [80]:
df.loc[(df.Account == 'FEDERATION OF FINNISH ENTERPRISES')]

,MasterID,AccountID,Account,Email domain,Acronym,[Bonus] Confidence score
399,205008.0,252198,FEDERATION OF FINNISH ENTERPRISES,@skal.fi,NaN,NaN
400,NaN,443307,FEDERATION OF FINNISH ENTERPRISES,@yrittajat.fi,NaN,NaN
